In [2]:
# Install Necessary Libraries
!pip install torchaudio==0.4.0 torch==1.4.0

     |████████████████████████████████| 3.1MB 2.7MB/s 
     |████████████████████████████████| 753.4MB 21kB/s 
ERROR: torchvision 0.6.1+cu101 has requirement torch==1.5.1, but you'll have torch 1.4.0 which is incompatible.
  Found existing installation: torch 1.5.1+cu101
    Uninstalling torch-1.5.1+cu101:
      Successfully uninstalled torch-1.5.1+cu101


In [16]:
# Import Necessary Libaries
import os
import torch
import torch.nn as nn
import torchaudio
import numpy as np
import time
from openpyxl import load_workbook

In [17]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# Unzip WAV zip file and store wav files onto disk
!unzip '/content/drive/My Drive/Colab Notebooks/Project/WAV Files.zip' -d '/root/WAV'

# Copy Validated_Samples.xlsx file onto disk
!cp '/content/drive/My Drive/Colab Notebooks/Project/Validated_Samples.xlsx' '/content'

Streaming output truncated to the last 5000 lines.
  inflating: /root/WAV/WAV Files/common_voice_en_667185.mp3.wav  
  inflating: /root/WAV/WAV Files/common_voice_en_667186.mp3.wav  
  inflating: /root/WAV/WAV Files/common_voice_en_667187.mp3.wav  
  inflating: /root/WAV/WAV Files/common_voice_en_667263.mp3.wav  
  inflating: /root/WAV/WAV Files/common_voice_en_667272.mp3.wav  
  inflating: /root/WAV/WAV Files/common_voice_en_667284.mp3.wav  
  inflating: /root/WAV/WAV Files/common_voice_en_667286.mp3.wav  
  inflating: /root/WAV/WAV Files/common_voice_en_667310.mp3.wav  
  inflating: /root/WAV/WAV Files/common_voice_en_667328.mp3.wav  
  inflating: /root/WAV/WAV Files/common_voice_en_667329.mp3.wav  
  inflating: /root/WAV/WAV Files/common_voice_en_667387.mp3.wav  
  inflating: /root/WAV/WAV Files/common_voice_en_667631.mp3.wav  
  inflating: /root/WAV/WAV Files/common_voice_en_667633.mp3.wav  
  inflating: /root/WAV/WAV Files/common_voice_en_667634.mp3.wav  
  inflating: /root/WAV/WA

In [6]:
# Load xlsx file containing path names of validated audio samples
samples = load_workbook ('/content/Validated_Samples.xlsx')

In [7]:
# Obtain original row index used in spectrogram code
np.random.seed(50)
lastrow = samples['Validated_Samples'].max_row
range_array = np.array(range(2,lastrow + 1))
np.random.shuffle(range_array)

In [10]:
# Convert WAV audio files into MelSpectrograms as np arrays and save to local computer

train_audio_transforms = nn.Sequential(
    torchaudio.transforms.MelSpectrogram(sample_rate=16000, n_mels=128),
    torchaudio.transforms.FrequencyMasking(freq_mask_param=30),
    torchaudio.transforms.TimeMasking(time_mask_param=100)
)

valid_audio_transforms = torchaudio.transforms.MelSpectrogram()

n = 2
time_length = []
for i in range_array:
    start_time = time.time()

    # Get name of audio file
    file_name = samples['Validated_Samples'].cell(row=i, column=2).value + ".wav"
    audio_path = os.path.join('/root/WAV/WAV Files', file_name)

    percent = int(n / lastrow * 100)

    waveform, sample_rate = torchaudio.load(audio_path, normalization=True)
    # os.makedirs('/content/WAV_to_np/train/')
    # os.makedir('/content/WAV_to_np/val/')
    # os.makedir('/content/WAV_to_np/test/')

    # Convert WAV file into MelSpectrogram
    if percent < 70: # For Training Samples
        spec = train_audio_transforms(waveform).squeeze(0).transpose(0, 1)
        spec_np = spec.numpy()
        np.save('/content/WAV_to_np/train/' + file_name, spec_np)
    else: # For Valid and Test Samples
        spec = valid_audio_transforms(waveform).squeeze(0).transpose(0, 1)
        spec_np = spec.numpy()
        if percent > 84:
            np.save('/content/WAV_to_np/test/' + file_name, spec_np)
        else:
            np.save('/content/WAV_to_np/val/' + file_name, spec_np)

    if n % 100 == 0:
        # Print progress
        end_time = time.time()
        time_length.append(end_time - start_time)
        predicted_time = int(((sum(time_length) / len(time_length))*(lastrow - n)) / 60)
        print(f'{percent}% Complete, Time left: {predicted_time} min(s)')

    # Count iteration
    n += 1

# Zip WAV_to_np folder and download it onto local machine
!zip -r /content/WAV_to_np.zip /content/WAV_to_np

!cp WAV_to_np.zip '/content/drive/My Drive/Colab Notebooks/Project/'
!ls -lt '/content/drive/My Drive/Colab Notebooks/Project/' 

## This will take some time and about 12 GB on your Google Drive. Make sure
## you have enough space on your Drive

0% Complete, Time left: 6 min(s)
0% Complete, Time left: 6 min(s)
0% Complete, Time left: 6 min(s)
1% Complete, Time left: 6 min(s)
1% Complete, Time left: 7 min(s)
1% Complete, Time left: 7 min(s)
2% Complete, Time left: 7 min(s)
2% Complete, Time left: 6 min(s)
2% Complete, Time left: 6 min(s)
3% Complete, Time left: 6 min(s)
3% Complete, Time left: 7 min(s)
3% Complete, Time left: 7 min(s)
4% Complete, Time left: 8 min(s)
4% Complete, Time left: 8 min(s)
4% Complete, Time left: 8 min(s)
5% Complete, Time left: 8 min(s)
5% Complete, Time left: 8 min(s)
5% Complete, Time left: 8 min(s)
6% Complete, Time left: 7 min(s)
6% Complete, Time left: 7 min(s)
6% Complete, Time left: 7 min(s)
7% Complete, Time left: 7 min(s)
7% Complete, Time left: 7 min(s)
7% Complete, Time left: 7 min(s)
8% Complete, Time left: 8 min(s)
8% Complete, Time left: 8 min(s)
8% Complete, Time left: 8 min(s)
9% Complete, Time left: 8 min(s)
9% Complete, Time left: 7 min(s)
9% Complete, Time left: 7 min(s)
10% Comple